# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [52]:
import pandas as pd
import requests

In [53]:
# 1) Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. 
# Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de 
# linhas. Há linhas duplicadas?

sinasc = pd.read_csv('SINASC_RO_2019.csv')

In [54]:
sinasc.head(10)

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
5,1,2516500.0,110001,1,33,5.0,1 a 3 anos,621005.0,2.0,1.0,...,5,6,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
6,1,2516500.0,110001,1,37,1.0,8 a 11 anos,999992.0,1.0,0.0,...,5,7,ATIVO,MUNIC,Novo Horizonte do Oeste,Rondônia,-11.70974,-61.99439,230.0,843.446
7,1,2516500.0,110001,1,27,2.0,12 anos ou mais,514120.0,0.0,0.0,...,5,8,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
8,1,2679477.0,110001,1,20,5.0,8 a 11 anos,621005.0,0.0,1.0,...,2,9,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
9,1,2679477.0,110001,1,30,2.0,1 a 3 anos,999992.0,1.0,1.0,...,5,10,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [55]:
# Conte o número de registros e o número de registros não duplicados da base. 

print(sinasc.shape) # número de registros
print(sinasc.duplicated().sum()) # número de registros não duplicados da base
# -> Não tem registros duplicados

(27028, 69)
0


In [56]:
sinasc.duplicated() # Não tem registros duplicados

0        False
1        False
2        False
3        False
4        False
         ...  
27023    False
27024    False
27025    False
27026    False
27027    False
Length: 27028, dtype: bool

In [57]:
sinasc.drop_duplicates().shape # Não tem registros duplicados

(27028, 69)

In [59]:
# 2) Conte o número de valores missing por variável

colunas_com_NaN = sinasc.loc[:, sinasc.isna().sum() > 0] # filtrando apenas as colunas que possuem dados faltantes
colunas_com_NaN.columns # exibindo as colunas com dados faltantes

Index(['CODESTAB', 'ESTCIVMAE', 'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO',
       'QTDFILMORT', 'GESTACAO', 'GRAVIDEZ', 'PARTO', 'HORANASC', 'SEXO',
       'APGAR1', 'APGAR5', 'RACACOR', 'IDANOMAL', 'CODANOMAL', 'DTRECORIGA',
       'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010', 'SERIESCMAE',
       'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES',
       'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM', 'CONSPRENAT',
       'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO', 'TPNASCASSI',
       'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1', 'munResLat',
       'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [60]:
# mostrando apenas as colunas que possuem dados faltantes e a quantidade de dados faltantes

colunas_com_NaN.isna().sum().sort_values()

munResArea        1
munResLon         1
munResLat         1
munResAlt         1
SEXO              4
TPDOCRESP        14
HORANASC         21
DTNASCMAE        40
PARTO            49
DTDECLARAC       52
TPNASCASSI       61
TPFUNCRESP       67
GRAVIDEZ         79
APGAR1           96
APGAR5          103
CODESTAB        115
ESCMAE2010      249
ESCMAEAGR1      249
TPAPRESENT      265
NATURALMAE      298
CODMUNNATU      298
CODUFNATU       298
ESCMAE          312
ESTCIVMAE       317
IDANOMAL        591
RACACOR         647
RACACORMAE      661
STCESPARTO      747
CONSPRENAT      930
STTRABPART      947
QTDGESTANT     1212
GESTACAO       1232
SEMAGESTAC     1238
TPMETESTIM     1238
QTDFILVIVO     1573
QTDPARTCES     1791
QTDPARTNOR     1879
QTDFILMORT     2098
MESPRENAT      2867
CODOCUPMAE     2907
DTULTMENST    10072
SERIESCMAE    12710
IDADEPAI      19421
CODANOMAL     26814
DTRECORIGA    27028
dtype: int64

In [61]:
# 3) Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis,
# muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. 
# São elas:

# ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

lista = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
lista

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [21]:
# 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas 
# principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. 
# Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. 
# Para esta seleção, conte novamente o número de linhas e o número de missings.


In [62]:
# contando a quantidade de linhas missing

sinasc['APGAR5'].isna().sum()

103

In [63]:
# removendo os registros não preenchidos

sinasc.dropna(subset=['APGAR5'], inplace=True)

In [64]:
# conferindo se foi removido os dados faltantes

sinasc['APGAR5'].isna().sum()

0

In [65]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. 
# Vamos assumir que o não preenchido é o mesmo que o código 9.

In [75]:
# Tem 315 faltantes

sinasc['ESTCIVMAE'].isnull().sum() 

0

In [67]:
# Substituindo faltantes por 9
sinasc['ESTCIVMAE'].fillna(9, inplace=True)
## Exibe mensagem de erro quando executo pela lista criada, mas substitui por 9. Se fizer no df não aparece a msg de erro

In [68]:
sinasc['ESTCIVMAE'].isnull().sum() # Conferindo a substituição de faltantes por 9

0

In [69]:
sinasc['CONSULTAS'].isnull().sum() # Não tem nenhum NaN

0

In [70]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.

In [71]:
sinasc['QTDFILVIVO'].isnull().sum()  # contando a quantidade de linhas missing

1566

In [72]:
sinasc['QTDFILVIVO'].fillna(0, inplace=True) # sustituindo NaN por 0

In [73]:
sinasc['QTDFILVIVO'].isnull().sum() # conferindo a substituição

0

In [78]:
# 7) Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' 
# como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, 
# e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

sinasc.isna().sum().sort_values()

ORIGEM            0
DTRECEBIM         0
VERSAOSIST        0
NUMEROLOTE        0
STDNEPIDEM        0
              ...  
DTULTMENST    10016
SERIESCMAE    12664
IDADEPAI      19332
CODANOMAL     26714
DTRECORIGA    26925
Length: 69, dtype: int64

In [79]:
# Verifico a quantidade de registros vazios na váriável escolhida
sinasc['CODANOMAL'].isna().sum()

26714

In [81]:
sinasc['CODANOMAL'].fillna(9, inplace=True) # Altero o valor vazio para 9 e confirmo a gravação do dado.
sinasc['CODANOMAL'].isnull().sum() #Na linha seguinte verifico se a quantidade de registros vazios ficou zerada.

0

In [82]:
# 8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:
# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.


In [83]:
# Crie uma categorização dessa variável com essa codificação

sinasc.loc[(sinasc['APGAR1'] >= 0) & (sinasc['APGAR1'] <= 3),
           'APGAR_CAT'] = 'ASFIXIA SEVERA'
sinasc.loc[(sinasc['APGAR1'] >= 4) & (sinasc['APGAR1'] <= 5),
           'APGAR_CAT'] = 'ASFIXIA MODERADA'
sinasc.loc[(sinasc['APGAR1'] >= 6) & (sinasc['APGAR1'] <= 7),
           'APGAR_CAT'] = 'ASFIXIA LEVE'
sinasc.loc[(sinasc['APGAR1'] >= 8) & (sinasc['APGAR1'] <= 10),
           'APGAR_CAT'] = 'NORMAL'

In [84]:
sinasc[['APGAR1', 'APGAR_CAT']]

,APGAR1,APGAR_CAT
0,9.0,NORMAL
1,8.0,NORMAL
2,8.0,NORMAL
3,9.0,NORMAL
4,8.0,NORMAL
...,...,...
27023,8.0,NORMAL
27024,8.0,NORMAL
27025,9.0,NORMAL
27026,8.0,NORMAL


In [85]:
# E calcule as frequências dessa categorização

pd.get_dummies(sinasc['APGAR_CAT'])

,ASFIXIA LEVE,ASFIXIA MODERADA,ASFIXIA SEVERA,NORMAL
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
27023,0,0,0,1
27024,0,0,0,1
27025,0,0,0,1
27026,0,0,0,1


In [86]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as 
# palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

In [87]:
# renomendo para letras minúsculas e incluindo underscore

sinasc['APGAR_CAT'] = sinasc['APGAR_CAT'].str.replace(' ', '_').str.lower() # transformando os valores em snake case

In [88]:
# exibindo a alteração

sinasc[['APGAR1', 'APGAR_CAT']]

,APGAR1,APGAR_CAT
0,9.0,normal
1,8.0,normal
2,8.0,normal
3,9.0,normal
4,8.0,normal
...,...,...
27023,8.0,normal
27024,8.0,normal
27025,9.0,normal
27026,8.0,normal
